# SUbvariant COmbine

In [97]:
from scipy import stats
import matplotlib.pyplot as plt
import pickle
from statsmodels.tsa.seasonal import seasonal_decompose
import pytz
from datetime import datetime
import pandas as pd
import numpy as np
from datetime import timedelta
def get_filterd_dataset(product_name):

    new_df = pd.read_csv('data/master/master_historical_till_2023_10_17.csv')
    # new_df = new_df.dropna(how='all')

    filtered = new_df[new_df['product_name']==product_name]
    filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])
    return filtered

def add_inflation(new_df):
    values = [
        3.68, 3.37, 3.14, 3.13, 3.16, 3.2, 3.12, 3.1, 3.23, 3.33, 3.27, 3.18, 3.15, 3.01, 3.21, 3.25, 3.31, 3.43, 3.79, 3.75,
        3.82, 3.86, 3.84, 3.8, 3.62, 3.72, 3.86, 3.51, 2.85, 2.19, 1.97, 1.88, 1.97, 1.75, 1.49, 1.61, 1.6, 1.56, 1.51, 1.95,
        3.3, 3.63, 3.97, 4.44, 4.51, 4.58, 5.26, 5.62, 6.94, 8.01, 8.53, 9.23, 9.07, 9.67, 10.21, 10.84, 11.44, 12.22, 12.53,
        13.12, 13.25, 13.28, 13.34, 12.82, 12.36, 12.13, 11.78, 11.43,10.79, 10.79
    ]

    # Create a list of dates from Jan 2018 to Aug 2023
    date_range = pd.date_range(start='2018-01-01', end='2023-10-30', freq='D')

    # Create a DataFrame

    inflation = []
    df = pd.DataFrame()
    df['date'] = date_range
    i = 0
    prev_month = df['date'][0].month
    # Populate the 'Price' column with values based on the corresponding month's value
    for date in df['date']:
        month = date.month
        if prev_month != month:
            prev_month = month
            i+=1
        inflation.append(values[i])
        # df.at[date, 'Price'] = value

    # Print the resulting DataFrame
    df['inflation'] = inflation
    # print(df)
    # df = df.set_index('date')
    df['date'] = pd.to_datetime(df['date'])
    new_df['date'] = pd.to_datetime(new_df['date'])
    # filtered2 = pd.merge(filtered2, df, how='left', left_index=True, right_index=True)
    merged = pd.merge(new_df, df, on='date', how='left')
    return merged

def generate_datafrane_for_profit2(date, price):
    test_date = pd.to_datetime(date)
    dates = [test_date.date()] * 24
    prices = [price] * 24
    hours = list(range(24))
    
 
    # Create DataFrame
    df = pd.DataFrame({'datetime': dates, 'hour': hours, 'net_price': prices})
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['date'] = df['datetime'].dt.date
    df['day_of_week_n'] = df['datetime'].dt.day_of_week

    for day_num in range(0, 7):
        day_col_name = f'net_price_day_{day_num}'
        df[day_col_name] = df.apply(lambda row: row['net_price'] if row['day_of_week_n'] == day_num else 0, axis=1)
    # merged_Df = add_inflation(df)[columns[:-1]]
    df['date'] = df['date'].astype(str)
    merged_Df = df
    
    
    return merged_Df.sort_values(['date','hour'])

def mode(x):
    return stats.mode(x)[0][0] 

def add_to_Historic_Dataframe(new_data,date):
  # new_data = pd.read_csv('data/Frubana_2023_10_09.csv')
  
  historical =  pd.read_csv('historic_pilot_info_catalan.csv')
  colombia_tz = pytz.timezone('America/Bogota')
    # Convert the 'order_submited_datetime' column to Colombia timezone
  historical['order_submited_datetime'] = pd.to_datetime(historical['order_submited_datetime'])
  historical['order_submited_datetime'] = historical['order_submited_datetime'].dt.tz_convert(colombia_tz)

  new_data['order_submited_datetime'] = pd.to_datetime(new_data['order_submited_datetime'])
  new_data['order_submited_datetime'] = new_data['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
  new_data['order_submited_date'] = new_data['order_submited_datetime'].dt.date.astype(str)
  
  new_data = new_data[new_data['order_submited_date']>date]

  common_columns = new_data.columns.intersection(historical.columns)
  merged_df = pd.concat([historical[common_columns], new_data[common_columns]])

  merged_df.sort_values('order_submited_datetime').to_csv(f'data/created/Catalan_{date}.csv')

  return merged_df.sort_values('order_submited_datetime')


def get_papa_grande():
    df = get_filterd_dataset('Champiñón Blanco Paris 500 gr Bandeja')
    colombia_tz = pytz.timezone('America/Bogota')

    # Convert the 'order_submited_datetime' column to Colombia timezone
    if df['order_submited_datetime'].dt.tz is None:
    # If the datetime is naive, localize it
        df['order_submited_datetime'] = df['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
    else:
        # If the datetime is already timezone-aware, just convert it
        df['order_submited_datetime'] = df['order_submited_datetime'].dt.tz_convert(colombia_tz)

    df['order_submited_datetime'] = pd.to_datetime(df['order_submited_datetime'])
    df['hour'] = df['order_submited_datetime'].dt.hour
    df['date'] = df['order_submited_datetime'].dt.date
    df['month'] = df['order_submited_datetime'].dt.month
    # df['day_of_week_n'] = df['order_submited_datetime'].dt.day_of_week
    df['day_of_month'] = df['order_submited_datetime'].dt.day
    df['year'] = df['order_submited_datetime'].dt.year
    agg_df = df.groupby(['date', 'hour']).agg({
    'net_price': mode,
    'quantity': 'sum',
    'cost': 'first',
    'avg_bench': 'first',
    'product_id': 'first',
    'product_name': 'first',
    'gross_price': 'first',
    # 'day_of_week_n': 'first',
    'day_of_month': 'first',
    # 'date': 'first'
    'year':'first',
    'month': 'first'

    }).reset_index()

    min_date = pd.to_datetime(agg_df['date']).min()
    max_date = pd.to_datetime(agg_df['date']).max()

    date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    hour_range = list(range(24))

    full_df = pd.DataFrame([(date, hour) for date in date_range for hour in hour_range], columns=['date', 'hour'])
    full_df['date'] = full_df['date'].dt.date.astype(str)  # Convert datetime to string to match df's 'date' format
    temp = agg_df[['date','hour']]
    temp_unique = temp.drop_duplicates()
    temp_unique['date'] = pd.to_datetime(temp_unique['date'])
    temp_unique['date'] = temp_unique['date'].dt.date.astype(str)
    # Identify missing rows
    merged_df = pd.merge(full_df, temp_unique, on=['date', 'hour'], how='left', indicator=True)
    missing_rows = merged_df[merged_df['_merge'] == 'left_only'][['date', 'hour']]
    # missing_rows.head(23)
    agg_df['date'] = pd.to_datetime(agg_df['date'])
    agg_df['date'] = agg_df['date'].dt.date.astype(str)
    merged_df.to_csv('temp.csv')
    
    merged_df = pd.merge(agg_df, missing_rows, on=['date', 'hour'], how='outer')
    merged_df.to_csv('temp3.csv')
    
    new_date = merged_df.iloc[-1]['date']
    new_price = merged_df.iloc[-1]['net_price']


    date_format = '%Y-%m-%d'
    date_DATE = datetime.strptime(new_date, date_format)
    # Add one day to the date
    next_date = date_DATE + timedelta(days=1)
    next_date_str = next_date.strftime(date_format)
    print('next date',next_date_str)
    test_vector_copy = generate_datafrane_for_profit2(next_date_str,new_price )
    print('new_test_vecotr',test_vector_copy.iloc[-1]['date'])
    merged_df = merged_df.sort_values(['date','hour'])
    merged_df = pd.concat([ merged_df, test_vector_copy])

    test_vector_copy = generate_datafrane_for_profit2('2023-10-19',new_price )
    print('new_test_vecotr',test_vector_copy.iloc[-1]['date'])
    merged_df = merged_df.sort_values(['date','hour'])
    merged_df = pd.concat([ merged_df, test_vector_copy])
    print('new_row',merged_df.iloc[-1]['date'])

    merged_df.to_csv('temp4.csv')
    
    print('new_row',merged_df.iloc[-1]['date'])


    cols_to_fill = [col for col in merged_df.columns if col not in ['quantity']]
    merged_df = merged_df.sort_values(['date','hour'])
    
    merged_df['net_price'] = merged_df['net_price'].fillna(method='ffill')
    merged_df['quantity'].fillna(0, inplace=True)
    merged_df.to_csv('temp5.csv')
    
    merged_df['net_price_FP'] = merged_df['net_price']
    merged_df['quantity_FP'] = merged_df['quantity']
    merged_df['net_price_FP_lag48'] = merged_df['net_price_FP'].shift(48)
    merged_df['net_price_FP_lag49'] = merged_df['net_price_FP'].shift(49)
    merged_df['quantity_FP_lag48'] = merged_df['quantity_FP'].shift(48)
    merged_df['quantity_FP_lag49'] = merged_df['quantity_FP'].shift(49)
    
    # if not comp
    daily_quantity_papagrande = merged_df.groupby('date').agg({'quantity': 'sum','net_price': mode}).reset_index()
    daily_quantity_papagrande.to_csv('temp1.csv')
    result = seasonal_decompose(daily_quantity_papagrande['quantity'], model='additive', period=7)
    daily_quantity_papagrande['FP_residual_daily'] = result.resid
    daily_quantity_papagrande['FP_trend_daily'] = result.trend
    daily_quantity_papagrande['FP_seasonal_daily'] = result.seasonal
    daily_quantity_papagrande['FP_residual_daily_lag2'] = daily_quantity_papagrande['FP_residual_daily'].shift(2)
    daily_quantity_papagrande['FP_trend_daily_lag2'] = daily_quantity_papagrande['FP_trend_daily'].shift(2)
    daily_quantity_papagrande['FP_seasonal_daily_lag2'] = daily_quantity_papagrande['FP_seasonal_daily'].shift(2)
   
    
    daily_quantity_papagrande['FP_rolling_avg_quantity_7'] = (
        daily_quantity_papagrande['quantity']
        .rolling(window=7)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity_papagrande['FP_rolling_avg_quantity_30'] = (
        daily_quantity_papagrande['quantity']
        .rolling(window=30)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity_papagrande['FP_quantity_daily_lag2'] = (
            daily_quantity_papagrande['quantity']
            .shift(2)  # This adds the 1-day lag
        )
    daily_quantity_papagrande['FP_net_price_daily_lag2'] = (
            daily_quantity_papagrande['net_price']
            .shift(2)  # This adds the 1-day lag
        )

    daily_quantity_papagrande['FP_net_price_daily_lag1'] = (
            daily_quantity_papagrande['net_price']
            .shift(1)  # This adds the 1-day lag
        )

    def ewm_features(dataframe, alphas, lags):
      for alpha in alphas:
          for lag in lags:
              dataframe['FP_quantity_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                  dataframe['quantity'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
      dataframe.fillna(0, inplace=True)
      return dataframe
    
    alphas = [0.95, 0.5]
    lags   = [2,7,10,14,30,60]


    daily_quantity_papagrande= ewm_features(daily_quantity_papagrande, alphas, lags)

    daily_quantity_papagrande['FP_daily_net_price'] = daily_quantity_papagrande['net_price']
    daily_quantity_papagrande['FP_daily_quantity'] = daily_quantity_papagrande['quantity']
    daily_quantity_papagrande['date']= daily_quantity_papagrande['date'].astype(str)
    merged_df['date']= merged_df['date'].astype(str)
    
    return daily_quantity_papagrande, merged_df



def aggregate_by_hour(df, time_bucket, has_competitor_price):
    """
    Aggregate a DataFrame by a specified time bucket (e.g., 24 for daily aggregation).
    
    Args:
        df (pd.DataFrame): Input DataFrame with a datetime field.
        time_bucket (int): Number of hours in each time bucket for aggregation.
    
    Returns:
        pd.DataFrame: Aggregated DataFrame with the datetime bucketed and values aggregated.
    """
    # Create a timezone object for Colombia (CST)
    colombia_tz = pytz.timezone('America/Bogota')
    
    # Convert the 'order_submited_datetime' column to Colombia timezone
    df['order_submited_datetime'] = df['order_submited_datetime'].dt.tz_convert(colombia_tz)

    df['order_submited_datetime'] = pd.to_datetime(df['order_submited_datetime'])
    df['hour'] = df['order_submited_datetime'].dt.hour
    df['date'] = df['order_submited_datetime'].dt.date
    df['month'] = df['order_submited_datetime'].dt.month
    # df['day_of_week_n'] = df['order_submited_datetime'].dt.day_of_week
    df['day_of_month'] = df['order_submited_datetime'].dt.day
    df['year'] = df['order_submited_datetime'].dt.year
    agg_df = df.groupby(['date', 'hour', 'net_price']).agg({
    'quantity': 'sum',
    'cost': 'first',
    'avg_bench': 'first',
    'product_id': 'first',
    'product_name': 'first',
    'gross_price': 'first',
    # 'day_of_week_n': 'first',
    'day_of_month': 'first',
    # 'date': 'first'
    'year':'first',
    'month': 'first'

}).reset_index()

    min_date = pd.to_datetime(agg_df['date']).min()
    max_date = pd.to_datetime(agg_df['date']).max()

    date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    hour_range = list(range(24))

    full_df = pd.DataFrame([(date, hour) for date in date_range for hour in hour_range], columns=['date', 'hour'])
    full_df['date'] = full_df['date'].dt.date.astype(str)  # Convert datetime to string to match df's 'date' format
    temp = agg_df[['date','hour']]
    temp_unique = temp.drop_duplicates()
    temp_unique['date'] = pd.to_datetime(temp_unique['date'])
    temp_unique['date'] = temp_unique['date'].dt.date.astype(str)
    # Identify missing rows
    merged_df = pd.merge(full_df, temp_unique, on=['date', 'hour'], how='left', indicator=True)
    missing_rows = merged_df[merged_df['_merge'] == 'left_only'][['date', 'hour']]
    # missing_rows.head(23)
    agg_df['date'] = pd.to_datetime(agg_df['date'])
    agg_df['date'] = agg_df['date'].dt.date.astype(str)
    merged_df = pd.merge(agg_df, missing_rows, on=['date', 'hour'], how='outer')
    new_date = merged_df.iloc[-1]['date']
    new_price = merged_df.iloc[-1]['net_price']


    date_format = '%Y-%m-%d'
    date_DATE = datetime.strptime(new_date, date_format)
    # Add one day to the date
    next_date = date_DATE + timedelta(days=1)
    next_date_str = next_date.strftime(date_format)
    print('next date',next_date_str)
    test_vector_copy = generate_datafrane_for_profit2(next_date_str,new_price )
    print('new_test_vecotr',test_vector_copy.iloc[-1]['date'])
    merged_df = merged_df.sort_values(['date','hour'])
    merged_df = pd.concat([ merged_df, test_vector_copy])
    print('new_row',merged_df.iloc[-1]['date'])

    test_vector_copy = generate_datafrane_for_profit2('2023-10-19',new_price )
    print('new_test_vecotr',test_vector_copy.iloc[-1]['date'])
    merged_df = merged_df.sort_values(['date','hour'])
    merged_df = pd.concat([ merged_df, test_vector_copy])
    print('new_row',merged_df.iloc[-1]['date'])


    cols_to_fill = [col for col in merged_df.columns if col not in ['quantity']]
    merged_df = merged_df.sort_values(['date','hour'])
    merged_df['net_price'] = merged_df['net_price'].fillna(method='ffill')
   
    
    
        
    merged_df['quantity'].fillna(0, inplace=True)

    # if not comp
    daily_quantity = merged_df.groupby('date').agg({'quantity': 'sum', 'net_price': 'mean'}).reset_index()


    result = seasonal_decompose(daily_quantity['quantity'], model='additive', period=7)
    daily_quantity['residual_daily'] = result.resid
    daily_quantity['trend_daily'] = result.trend
    daily_quantity['seasonal_daily'] = result.seasonal
    daily_quantity['residual_daily_lag2'] = daily_quantity['residual_daily'].shift(2)
    daily_quantity['trend_daily_lag2'] = daily_quantity['trend_daily'].shift(2)
    daily_quantity['seasonal_daily_lag2'] = daily_quantity['seasonal_daily'].shift(2)
   
    #  if  comp
    # daily_quantity = merged_df.groupby('date').agg({'quantity': 'sum', 'net_price': 'mean','avg_bench':'mean'}).reset_index()

    # Step 2: Calculate 7-Day Rolling Average with 1-day Lag
    daily_quantity['rolling_avg_quantity_7_lag2'] = (
        daily_quantity['quantity']
        .rolling(window=7)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['rolling_avg_quantity_10_lag2'] = (
        daily_quantity['quantity']
        .rolling(window=10)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['rolling_avg_quantity_15_lag2'] = (
        daily_quantity['quantity']
        .rolling(window=15)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['rolling_avg_quantity_30_lag2'] = (
        daily_quantity['quantity']
        .rolling(window=30)
        .mean()
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['quantity_daily_lag2'] = (
        daily_quantity['quantity']
        .shift(2)  # This adds the 1-day lag
    )

    daily_quantity['ratio_rolling_avg_quantity_30_7_lag2'] = daily_quantity['rolling_avg_quantity_30_lag2']/daily_quantity['rolling_avg_quantity_7_lag2']
    

    def ewm_features(dataframe, alphas, lags):
      for alpha in alphas:
          for lag in lags:
              dataframe['quantity_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                  dataframe['quantity'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
      dataframe.fillna(0, inplace=True)
      return dataframe
    
    alphas = [0.95, 0.5]
    lags   = [2,7,10,14,30,60]


    daily_quantity= ewm_features(daily_quantity, alphas, lags)


    # daily_quantity['diff'] = daily_quantity['net_price'] - daily_quantity['avg_bench']
    # daily_quantity['avg_bench_lag1'] = daily_quantity['avg_bench'].shift(1)
    # daily_quantity['diff_cross'] = daily_quantity['net_price'] - daily_quantity['avg_bench_lag1']

    # daily_quantity['diff_lag2'] = (
    #     daily_quantity['diff']
    #     .shift(2)
    #  # This adds the 1-day lag
    # )
    # daily_quantity['diff_cross_lag2'] = (
    #     daily_quantity['diff_cross']
    #     .shift(2)
    #  # This adds the 1-day lag
    # )

    # merged_df = merged_df.merge(daily_quantity[['date', 'rolling_avg_quantity_30','rolling_avg_quantity_7','quantity_daily_lag2']], on='date', how='left')
    
    merged_df = merged_df.merge(daily_quantity[['date', 'ratio_rolling_avg_quantity_30_7_lag2','quantity_ewm_alpha_095_lag_2', 'quantity_ewm_alpha_095_lag_7','quantity_ewm_alpha_095_lag_10','quantity_ewm_alpha_095_lag_14'
                                                ,'quantity_ewm_alpha_095_lag_30','quantity_ewm_alpha_095_lag_60','quantity_ewm_alpha_05_lag_2', 'quantity_ewm_alpha_05_lag_7','quantity_ewm_alpha_05_lag_10','quantity_ewm_alpha_05_lag_14'
                                                ,'quantity_ewm_alpha_05_lag_30','quantity_ewm_alpha_05_lag_60',
                                                'rolling_avg_quantity_30_lag2','rolling_avg_quantity_15_lag2','rolling_avg_quantity_10_lag2','rolling_avg_quantity_7_lag2','quantity_daily_lag2','trend_daily_lag2','seasonal_daily_lag2','residual_daily_lag2']], on='date', how='left')
    
    daily_quantity_FP, hourly_FP = get_papa_grande()
    merged_df = merged_df.merge(daily_quantity_FP[['date','FP_quantity_ewm_alpha_095_lag_7','FP_trend_daily_lag2','FP_net_price_daily_lag1','FP_net_price_daily_lag2', 'FP_quantity_daily_lag2','FP_rolling_avg_quantity_30','FP_rolling_avg_quantity_7','FP_daily_quantity','FP_daily_net_price']], on='date', how='left')

    merged_df = merged_df.merge(hourly_FP[['date','hour','net_price_FP_lag48','quantity_FP_lag48','net_price_FP_lag49','quantity_FP_lag49','net_price_FP','quantity_FP']], on=['date', 'hour'], how='left')

    merged_df = merged_df.sort_values(['date','hour'])
    merged_df['date']=merged_df['date'].astype(str)
    
    merged_df = merged_df.sort_values(['date','hour'])
    
    return merged_df



def create_features(dataframe, has_competitor_price):
    def is_weekend(day):
        return 1 if day in [5, 6] else 0
    
    # dataframe['datetime'] = pd.to_datetime(dataframe['date'])
    
    cols_to_fill = [col for col in dataframe.columns if col not in ['quantity']]
    dataframe = dataframe.sort_values(['date','hour'])
    # dataframe['net_price'] = dataframe['net_price'].fillna(method='ffill')
    dataframe['quantity'].fillna(0, inplace=True)

    dataframe = add_inflation(dataframe)
    dataframe['day_of_week_n'] = dataframe['date'].dt.day_of_week

    dataframe['if_weekend'] = dataframe['day_of_week_n'].apply(is_weekend)
    for day_num in range(0, 7):
        day_col_name = f'net_price_day_{day_num}'
        dataframe[day_col_name] = dataframe.apply(lambda row: row['net_price'] if row['day_of_week_n'] == day_num else 0, axis=1)

    print('features ', dataframe.info())
    return dataframe


In [98]:
time_bucket=24
has_competitor_price=1

filtered1 = get_filterd_dataset('Champiñón Blanco Paris 1Kg Bandeja')

daily_aggregated_df = aggregate_by_hour(filtered1, time_bucket=time_bucket, has_competitor_price=has_competitor_price)
features = create_features(daily_aggregated_df, has_competitor_price)

features = features.sort_values(['date','hour'])

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_10583/757281885.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])
/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_10583/757281885.py:315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_unique['date'] = pd.to_datetime(temp_unique['date'])
/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_10583/757281885.py:316: SettingWithCopyWarning: 

next date 2023-10-18
new_test_vecotr 2023-10-18
new_row 2023-10-18
new_test_vecotr 2023-10-19
new_row 2023-10-19


/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_10583/757281885.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['order_submited_datetime'] = pd.to_datetime(filtered['order_submited_datetime'])
/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_10583/757281885.py:80: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return stats.mode(x)[0][0]


next date 2023-10-18
new_test_vecotr 2023-10-18
new_test_vecotr 2023-10-19
new_row 2023-10-19
new_row 2023-10-19


/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_10583/757281885.py:80: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return stats.mode(x)[0][0]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18186 entries, 0 to 18185
Data columns (total 59 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   date                                  18186 non-null  datetime64[ns]
 1   hour                                  18186 non-null  int64         
 2   net_price                             18177 non-null  float64       
 3   quantity                              18186 non-null  float64       
 4   cost                                  6068 non-null   float64       
 5   avg_bench                             6529 non-null   float64       
 6   product_id                            6557 non-null   float64       
 7   product_name                          6557 non-null   object        
 8   gross_price                           6557 non-null   float64       
 9   day_of_month                          6557 non-null   float64       
 10

In [99]:
features[[
    'date', 'net_price', 'day_of_week_n', 'net_price_day_0', 'net_price_day_1',
    'net_price_day_2', 'net_price_day_3', 'net_price_day_4', 'net_price_day_5',
    'net_price_day_6', 'hour', 'quantity_ewm_alpha_095_lag_7',
    'FP_quantity_daily_lag2', 'trend_daily_lag2', 'quantity'
]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18186 entries, 0 to 18185
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   date                          18186 non-null  datetime64[ns]
 1   net_price                     18177 non-null  float64       
 2   day_of_week_n                 18186 non-null  int32         
 3   net_price_day_0               18177 non-null  float64       
 4   net_price_day_1               18186 non-null  float64       
 5   net_price_day_2               18186 non-null  float64       
 6   net_price_day_3               18186 non-null  float64       
 7   net_price_day_4               18186 non-null  float64       
 8   net_price_day_5               18186 non-null  float64       
 9   net_price_day_6               18186 non-null  float64       
 10  hour                          18186 non-null  int64         
 11  quantity_ewm_alpha_095_lag_7

In [96]:
features.to_csv('combined_data/CBP1kg_500(mode).csv')

In [53]:
pd.read_csv('combined_data/PapaSabaneraSM_PCSM(mode).csv')

/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_10583/1689469489.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('combined_data/PapaSabaneraSM_GG(mode).csv')


,Unnamed: 0,date,hour,net_price,quantity,cost,avg_bench,product_id,product_name,gross_price,...,FP_daily_quantity,FP_daily_net_price,net_price_FP_lag48,quantity_FP_lag48,net_price_FP_lag49,quantity_FP_lag49,net_price_FP,quantity_FP,inflation,if_weekend
0,0,2021-10-17,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,7.5,3450.0,NaN,NaN,NaN,NaN,3450.0,0.0,4.58,1
1,1,2021-10-17,1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,7.5,3450.0,NaN,NaN,NaN,NaN,3450.0,0.0,4.58,1
2,2,2021-10-17,2,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,7.5,3450.0,NaN,NaN,NaN,NaN,3450.0,0.0,4.58,1
3,3,2021-10-17,3,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,7.5,3450.0,NaN,NaN,NaN,NaN,3450.0,0.0,4.58,1
4,4,2021-10-17,4,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,7.5,3450.0,NaN,NaN,NaN,NaN,3450.0,0.0,4.58,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17683,17683,2023-10-19,19,5000.8,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,5400.0,5400.0,17.5,5400.0,12.5,5400.0,0.0,10.79,0
17684,17684,2023-10-19,20,5000.8,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,5400.0,5400.0,0.0,5400.0,17.5,5400.0,0.0,10.79,0
17685,17685,2023-10-19,21,5000.8,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,5400.0,5400.0,0.0,5400.0,0.0,5400.0,0.0,10.79,0
17686,17686,2023-10-19,22,5000.8,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,5400.0,5400.0,0.0,5400.0,0.0,5400.0,0.0,10.79,0


# Create Master Dataset

In [10]:
# df_16 = pd.read_csv('data/Frubana_2023_10_16.csv')
# colombia_tz = pytz.timezone('America/Bogota')
 
# df_16['order_submited_datetime'] = pd.to_datetime(df_16['order_submited_datetime'])
# df_16['order_submited_datetime'] = df_16['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
# df_16['order_submited_date'] = df_16['order_submited_datetime'].dt.date.astype(str)
# df_16 = df_16[df_16['order_submited_date']<'2023-10-16']
# df_16.to_csv('data/master/master_historical.csv')


In [28]:
import pandas as pd

# Load the dataframes
df_17 = pd.read_csv('data/Frubana_2023_10_18.csv')
df_16 = pd.read_csv('data/master/master_historical_till_2023_10_16.csv')



In [29]:

# Display the first few rows of each dataframe
df_17_head = df_17.head()

colombia_tz = pytz.timezone('America/Bogota')
    # Convert the 'order_submited_datetime' column to Colombia timezone
df_17['order_submited_datetime'] = pd.to_datetime(df_17['order_submited_datetime'])
df_17['order_submited_datetime'] = df_17['order_submited_datetime'].dt.tz_localize(pytz.utc).dt.tz_convert(colombia_tz)
df_17['order_submited_date'] = df_17['order_submited_datetime'].dt.date.astype(str)
df_17 = df_17[df_17['order_submited_date']<'2023-10-18']
df_17 = df_17[df_17['order_submited_date']>'2023-10-16']

combined_df = pd.concat([df_17, df_16], ignore_index=True)
combined_df.to_csv('data/master/master_historical_till_2023_10_17.csv')

In [26]:
data = pd.read_csv('data/master/master_historical_till_2023_10_17_temp.csv')
data.sort_values('order_submited_datetime')

,Unnamed: 0.2,region_code,order_submited_date,order_submited_datetime,parent_category,category,product_id,product_name,cost,benchmark,median_bench,avg_bench,max_bench,gross_price,net_price,quantity,product_step_unit,benchmark_count,Unnamed: 0.1,Unnamed: 0
22826,22826,BOG,2021-10-16,2021-10-16 19:04:00-05:00,Frutas & verduras,Verduras,-882794291467029550,Pimentón Maduración Mixta Semi (Mediano / Nata...,NaN,NaN,NaN,NaN,NaN,2822.00,2116.50,1.5,1.5,NaN,22015.0,21291.0
215856,215856,BOG,2021-10-16,2021-10-16 19:06:00-05:00,Frutas & verduras,Tubérculos,8673176760259137473,Papa Sabanera Gruesa (Grande) Kg,NaN,NaN,NaN,NaN,NaN,3450.00,3450.00,2.5,2.5,NaN,215045.0,214378.0
313146,313146,BOG,2021-10-16,2021-10-16 19:24:00-05:00,Frutas & verduras,Verduras,-882794291467029550,Pimentón Maduración Mixta Semi (Mediano / Nata...,NaN,NaN,NaN,NaN,NaN,2822.00,2116.50,1.5,1.5,NaN,312335.0,311708.0
94578,94578,BOG,2021-10-16,2021-10-16 19:45:00-05:00,Frutas & verduras,Verduras,-882794291467029550,Pimentón Maduración Mixta Semi (Mediano / Nata...,NaN,NaN,NaN,NaN,NaN,2822.00,2116.50,3.0,1.5,NaN,93767.0,93061.0
22820,22820,BOG,2021-10-16,2021-10-16 19:48:00-05:00,Frutas & verduras,Tubérculos,-3386165639547488916,Papa Criolla Lavada Semi (Mediana) Kg,NaN,NaN,NaN,NaN,NaN,3920.00,3920.00,4.0,1.0,NaN,22009.0,21285.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,115,BOG,2023-10-17,2023-10-17 23:06:00-05:00,Frutas & verduras,Verduras,-6536998881607504388,Apio Estándar (Grande) Atado,615.0477,1200.0,1200.0,1200.0,1200.0,1100.00,1034.00,15.0,5.0,1.0,NaN,NaN
23,23,BOG,2023-10-17,2023-10-17 23:10:00-05:00,Frutas & verduras,Tubérculos,-8910485076695139605,Papa Criolla Segunda Mixta Kg,1614.5149,2418.0,2418.0,2418.0,2418.0,2680.00,2519.20,10.0,10.0,1.0,NaN,NaN
244,244,BOG,2023-10-17,2023-10-17 23:10:00-05:00,Frutas & verduras,Verduras,-882794291467029550,Pimentón Maduración Mixta Semi (Mediano / Nata...,1821.8443,4000.0,4000.0,4000.0,4000.0,3730.00,3730.00,2.0,1.0,1.0,NaN,NaN
665,665,BOG,2023-10-17,2023-10-17 23:17:00-05:00,Frutas & verduras,Verduras,7749572820280257375,Cebolla Cabezona Roja Sin Pelar Tamaño Mixto D...,4345.9175,2292.0,2292.0,2292.0,2292.0,2639.92,2481.52,3.0,1.0,2.0,NaN,NaN
